Implementación y explicación del método para el preprocesado del texto.

Implementa una función llamada preprocess_post(text: str) en el archivo core.py.
La función debe limpiar y normalizar los textos de los posts (columna "post") y guardar el resultado en una nueva columna "clean_post".
Describe detalladamente los pasos de preprocesamiento en un notebook de Python (implementacion_modulo_1.ipynb), como eliminación de signos de puntuación, conversión a minúsculas, etc.

---**Importamos el dataframe**--- 

Utilizado posteriormente para columna 'clean_post' y creación del dataset para los siguientes módulos.

In [59]:
import pandas as pd

reddit_df = pd.read_csv('reddit_database_sentiment/reddit_database_sentiment.csv', delimiter=';', quotechar='"', encoding='utf-8', low_memory=False)

**Análisis de la información de la columna post**

In [60]:
print(reddit_df['post'][1])

I'm cross posting this from /r/cyberlaw, hopefully you guys find it as interesting as I did(it deals with Google Analytics):

So quite awhile ago, I ordered a Papa John's pizza online. My job largely involves looking at ads that appear online, so afterwards I was quick to notice *I was getting a LOT* of Papa Johns ads (especially at night) being served through a Google owned company (DoubleClick media). Yesterday one of these ads popped up again on Youtube (a place that typically serves using the adwords program, not doubleclick), so I decided to copy the URL. 

For those not in the advertising field: Making full use of Google's analytics tool means that certain information about the advertising campaign is leaked in the URL.

So let's break it apart: 

&gt;http://ad.doubleclick.net/click;h=(junk here);~sscs=?http://googleads.g.doubleclick.net/aclk?sa=l&amp;ai=(junk here)&amp;adurl=http://www.papajohns.com/index.shtm?utm_source=googlenetwork&amp;utm_medium=DisplayCPC&amp;utm_campaign=G

Aquí se observa que es necesario:
- Cambiar a minúsculas.
- Quitar saltos de líneas y demás letras escapadas ('\n', '\r', '\t', '\b', '\f').
- Eliminar URLs, que no aportan al análisis de sentimiento.
- Eliminar espacios extra.
- Eliminar caracteres especiales y números.
- Decodificar el html.

In [49]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

**Descargas necesarias (descomentar si no se tienen)**


In [50]:
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

**Explicación paso a paso**

Conversión a minúsculas: \
 Al convertir todo el texto a minúsculas, evitamos duplicados en diferentes casos (por ejemplo, "Data" y "data" se tratan como la misma palabra).
 ```python
    text = text.lower()	

Eliminación de URL's: \
Las URLs generalmente no aportan información relevante para el análisis de contenido y pueden distraer al modelo.
  ```python
    text = re.sub(r'http\S+|https\S+|www\S+', '', text, flags=re.MULTILINE)

Eliminación de menciones de usuario y subreddits: \
Las menciones específicas como @usuario o /r/subreddit son específicas y pueden no ser útiles para la clasificación general.
```python 
text = re.sub(r'@\w+|\/r\/\w+', '', text)

Eliminación de caracteres especiales y números: \
```python
text = text.translate(str.maketrans('', '', string.punctuation))


Eliminación de Números: \
Los números no pueden ser relevantes para el análisis de sentimiento. (Excluyendo en casos de notas o calificaciones)
```python
text = re.sub(r'\d+', '', text)

Tokenización, stopwords: \
Dividir el texto en tokens (palabras) facilita el procesamiento posterior, las stopwords son marcas gramaticales que no aportan al análisis de sentimiento.
```python
tokens = text.split()
stop_words = set(stopwords.words('english'))
tokens = [word for word in tokens if word not in stop_words]

En primera instancia, se buscaba lematizar el texto para quedarnos solo con la raíz de las palabras, aunque finalmente se decidió no hacerlo para no perder información relevante.


**Código completo de la función**

In [66]:
import string

lemmatizer = WordNetLemmatizer()

def preprocess_post(text: str) -> str:
    """
    Limpia y normaliza el texto de un post de Reddit.

    Parámetros:
    text (str): Texto original del post.

    Retorna:
    str: Texto normalizado para análisis.
    """

    text = str(text)

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    text = re.sub(r'@\w+|\/r\/\w+', '', text)
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    text = re.sub(r'\d+', '', text)
    
    text = text.lower()

    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    
    clean_text = ' '.join(lemmas)
    
    return clean_text

In [62]:
print(reddit_df.head())

          created_date created_timestamp  subreddit  \
0  2010-02-11 19:47:22      1265910442.0  analytics   
1  2010-03-04 20:17:26      1267726646.0  analytics   
2  2011-01-06 04:51:18      1294282278.0  analytics   
3  2011-01-19 11:45:30      1295430330.0  analytics   
4  2011-01-19 21:52:28      1295466748.0  analytics   

                                               title            author  \
0  So what do you guys all do related to analytic...              xtom   
1  Google's Invasive, non-Anonymized Ad Targeting...              xtom   
2  DotCed - Functional Web Analytics - Tagging, R...            dotced   
3            Program Details - Data Analytics Course     iqrconsulting   
4  potential job in web analytics... need to anal...  therewontberiots   

   author_created_utc                                          full_link  \
0        1.227476e+09  https://www.reddit.com/r/analytics/comments/b0...   
1        1.227476e+09  https://www.reddit.com/r/analytics/comments/b9...

**Prueba función** \
Se prueba la función con un texto de ejemplo para verificar que el preprocesamiento se realizó correctamente.

In [52]:
test_text = """
Check out this amazing project on machine learning! Visit https://github.com/user/repo for more details.
Thanks @datascientist for the insights. Also, shoutout to /r/MachineLearning for the support.
Contact me at 123-456-7890. #datascience #machinelearning
"""
clean_text = preprocess_post(test_text)
print("Texto original: ", test_text)
print("Texto limpio: ", clean_text)


Texto original:  
Check out this amazing project on machine learning! Visit https://github.com/user/repo for more details.
Thanks @datascientist for the insights. Also, shoutout to /r/MachineLearning for the support.
Contact me at 123-456-7890. #datascience #machinelearning

Texto limpio:  check amazing project machine learning visit detail thanks insight also shoutout support contact datascience machinelearning


**Creación de la columna clean_post y guardado del dataframe**
Ahora trabajaremos para adecuar el dataframe su totalidad, creando la columna clean_post y guardando el dataframe en un nuevo archivo csv.
```python

**Preparación del dataset**
1. Normalizamos las fechas del dataset en caso de que sean necesarias en posteriores módulo, mejorando la calidad de la información.
```python
reddit_df['created_date'] = pd.to_datetime(reddit_df['created_date'], format='%Y-%m-%d %H:%M', errors='coerce')
reddit_df['author_created_date'] = pd.to_datetime(reddit_df['author_created_utc'], unit='s', errors='coerce')



2. Aplicamos la función preprocess_post a la columna post.
```python
reddit_df['clean_post'] = reddit_df['post'].apply(preprocess_post)

In [63]:
reddit_df['created_date'] = pd.to_datetime(reddit_df['created_date'], format='%Y-%m-%d %H:%M', errors='coerce')
reddit_df['author_created_date'] = pd.to_datetime(reddit_df['author_created_utc'], unit='s', errors='coerce')

In [64]:
# print(reddit_df['post'].head())

0    There's a lot of reasons to want to know all t...
1    I'm cross posting this from /r/cyberlaw, hopef...
2    DotCed,a Functional Analytics Consultant, offe...
3    Here is the program details of the data analyt...
4    i decided grad school (physics) was not for me...
Name: post, dtype: object


<h1>Solución de problemas y optimización de la función.</h1>

Al aplicar la función preprocess llegábamos a un dataframe vacío, y el ritmo de procesamiento de las 274000 filas habría sido muy lento. Por ello, decidimos incluir mejoras para la función (como reducir las veces que se compila cada patrón regex de 1 vez por fila a 1 vez en total), así como para la aplicación en el dataframe.

<h2>Mejoras implementadas</h2>

-  Importaciones Optimizadas: usamos tqdm.auto e funciones específicas de NLTK para reducir overhead.

- Precompilación de Patrones Regex: mejora significativa en velocidad para grandes volúmenes de datos.

- Inicialización de Recursos NLTK: carga única de recursos y uso de set() para búsqueda O(1) de stopwords.

- Función Principal Optimizada: validación temprana y encadenamiento de operaciones.

- Procesamiento por Chunks: mejor gestión de memoria y progreso visible.

- Implementación con Control: manejo de errores y métricas de rendimiento.

- Estas mejoras han resultado capitales para solucionar nuestros problemas y crear 'processed_dataset' en mucho menos tiempo.

In [74]:
import time
from tqdm.auto import tqdm
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd

# 1. Precompilar patrones regex (más eficiente)
HTML_PATTERN = re.compile(r'<[^>]*>')
URL_PATTERN = re.compile(r'https?://\S+|www\.\S+')
MENTION_PATTERN = re.compile(r'@\w+|#\w+')
SPECIAL_CHARS = re.compile(r'[^\w\s]')

# 2. Cargar recursos NLTK una sola vez
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def optimized_preprocess(text: str) -> str:
    """Versión optimizada de preprocess_post"""
    if pd.isna(text):
        return ""
    
    # Aplicar todas las limpiezas de texto en una sola pasada
    text = (HTML_PATTERN.sub(' ', text.lower())
            .replace('\n', ' ')
            .replace('\t', ' '))
    
    text = URL_PATTERN.sub('', text)
    text = MENTION_PATTERN.sub('', text)
    text = SPECIAL_CHARS.sub(' ', text)
    
    # Tokenizar y limpiar en una sola pasada
    tokens = [
        lemmatizer.lemmatize(token)
        for token in word_tokenize(text)
        if token not in stop_words and token.isalnum()
    ]
    
    return ' '.join(tokens)

# 3. Procesar en chunks para mejor manejo de memoria
def process_in_chunks(df, chunk_size=10000):
    results = []
    for i in tqdm(range(0, len(df), chunk_size)):
        chunk = df[i:i + chunk_size].copy()
        chunk['clean_post'] = chunk['post'].swifter.apply(optimized_preprocess)
        results.append(chunk)
    return pd.concat(results)

# 4. Implementación con control de progreso y manejo de errores
try:
    print(f"Iniciando procesamiento de {len(reddit_df)} filas...")
    start_time = time.time()
    
    # Procesar en chunks
    processed_df = process_in_chunks(reddit_df)
    
    end_time = time.time()
    processing_time = (end_time - start_time) / 60
    
    print(f"""
    Procesamiento completado:
    - Tiempo total: {processing_time:.2f} minutos
    - Velocidad: {len(reddit_df)/processing_time:.0f} filas/minuto
    - Filas procesadas: {len(processed_df)}
    """)
    
    # Guardar resultados periódicamente
    processed_df.to_csv('processed_dataset.csv', 
                       index=False, 
                       sep=';',
                       encoding='utf-8')
    
except Exception as e:
    print(f"Error durante el procesamiento: {str(e)}")

Iniciando procesamiento de 274239 filas...


100%|██████████| 28/28 [02:37<00:00,  5.64s/it]



    Procesamiento completado:
    - Tiempo total: 2.63 minutos
    - Velocidad: 104158 filas/minuto
    - Filas procesadas: 274239
    


In [75]:
import zipfile

# Generar el archivo CSV
processed_dataset_path = 'processed_dataset.csv'

# Crear un ZIP y añadir el CSV
zip_file_name = 'processed_dataset.zip'
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(processed_dataset_path, arcname='processed_dataset.csv')